# MH of older workers, retirement age and working conditions

## Preprocessing EWCS data

Load libraries

In [1]:
import os
import sys

src_path = os.path.abspath("../")
sys.path.append(src_path)

from utils.common import *
from utils.ewcs import *

import_libraries()
import pyreadstat

Load data

In [2]:
ewcs, meta_ewcs = pyreadstat.read_dta(
    "/Users/alexandralugova/Documents/GitHub/MH-old-workers/data/datasets/EWCS 1991-2015 UKDA ISCO.DTA"
)
df10, meta_df10 = pyreadstat.read_dta(
    "/Users/alexandralugova/Documents/GitHub/MH-old-workers/data/datasets/UKDA-2010/stata/stata11/ewcs_2010_version_ukda_6_dec_2011.dta"
)

Apply some initial formatting

In [3]:
ewcs = ewcs_preprocessing(ewcs, meta_ewcs)

Calculate social environment index

In [4]:
soc = social_environment_index(ewcs, df10)
ewcs = ewcs.merge(soc, on=["id", "year"], how="left")

JQI social environment
        count       mean        std        min        25%        50%  \
year                                                                   
2010  30836.0  82.983566  14.530474  12.500000  75.000000  87.500000   
2015  13279.0  79.869788  15.768143   9.090909  72.727273  81.818182   

            75%    max  
year                    
2010  93.750000  100.0  
2015  90.909091  100.0  


Calculate prospects index

In [5]:
pro = prospects_index(ewcs, df10)
ewcs = ewcs.merge(pro, on=["id", "year"], how="left")

JQI prospects
        count       mean        std  min   25%   50%   75%    max
year                                                             
2010  38556.0  55.655086  23.024819  0.0  37.5  50.0  75.0  100.0
2015  30073.0  57.977255  24.018895  0.0  50.0  62.5  75.0  100.0


Calculate intensity index (greater index - lower intensity, inverse to original)

In [6]:
int = intensity_index(ewcs, df10)
ewcs = ewcs.merge(int, on=["id", "year"], how="left")

JQI intensity
        count       mean        std       min        25%        50%  \
year                                                                  
2010  27063.0  56.353951  15.846348  4.651163  46.511628  58.139535   
2015  12947.0  46.274705  13.852596  2.325581  37.209302  46.511628   

            75%        max  
year                        
2010  67.441860  95.348837  
2015  55.813953  83.720930  


Adjust indexes with design weights

In [7]:
indexes = [col for col in ewcs.columns if col.startswith("jqi")]

for index in indexes:
    ewcs[f"{index}_weighted"] = ewcs[index] * ewcs["w4"]

Calculate overall work quality index, normal and weighted

In [8]:
ewcs = sum_wq_index(ewcs)

JQI working quality index
       count        mean        std         min         25%         50%  \
year                                                                      
2010  8692.0  422.290177  58.719339  181.754106  386.940634  428.266737   
2015  5234.0  415.680452  61.528587  133.185597  379.178912  422.432728   

             75%         max  
year                          
2010  464.941052  575.941286  
2015  460.123278  555.676416  


Apply some final formatting

In [9]:
indexes = [col for col in ewcs.columns if col.startswith("jqi")]
ewcs = ewcs[["country", "year", "isco"] + indexes]  # choose columns

# 4 digits
ewcs_4digits = (
    ewcs.groupby(["country", "year", "isco"]).mean().reset_index(drop=False)
)  # aggregate on year-country-isco level
ewcs_4digits = ewcs_4digits.dropna().reset_index(drop=True)  # drop missing values

# 3 digits
ewcs["isco"] = ewcs["isco"] // 10
ewcs_3digits = (
    ewcs.groupby(["country", "year", "isco"]).mean().reset_index(drop=False)
)  # aggregate on year-country-isco level
ewcs_3digits = ewcs_3digits.dropna().reset_index(drop=True)  # drop missing values

Interpolate values for indexes for 2013

In [10]:
ewcs_4digits = interpolate2013(ewcs_4digits)
ewcs_3digits = interpolate2013(ewcs_3digits)

Final steps

In [11]:
ewcs_4digits["year"] = ewcs_4digits["year"].replace(
    {2010: 2011}
)  # change year to merge with SHARE
ewcs_3digits["year"] = ewcs_3digits["year"].replace({2010: 2011})

In [12]:
ewcs_4digits.to_csv(
    "/Users/alexandralugova/Documents/GitHub/MH-old-workers/data/datasets/results/work_quality_indexes_year_country_4digits.csv",
    index=False,
)  # save resulting dataset

ewcs_3digits.to_csv(
    "/Users/alexandralugova/Documents/GitHub/MH-old-workers/data/datasets/results/work_quality_indexes_year_country_3digits.csv",
    index=False,
)  # save resulting dataset

In [13]:
ewcs_4digits

,country,year,isco,jqi_monthly_earnings,jqi_skills_discretion,jqi_physical_environment,jqi_working_time_quality,jqi_social_environment,jqi_prospects,jqi_intensity,jqi_monthly_earnings_weighted,jqi_skills_discretion_weighted,jqi_physical_environment_weighted,jqi_working_time_quality_weighted,jqi_social_environment_weighted,jqi_prospects_weighted,jqi_intensity_weighted,jqi_sum,jqi_sum_weighted
0,Austria,2011,1114,2366.148112,77.290234,97.863248,100.0,100.0,75.0,52.325581,3034.577207,94.40666,121.392346,123.996761,60.31181,45.233857,47.096205,523.975161,316.018902
1,Austria,2011,1120,3143.596761,83.616787,85.042736,53.125,87.5,78.571429,53.820598,1638.952541,52.464895,53.989173,32.531523,47.019517,49.624036,34.3123,462.6496,250.798543
2,Austria,2011,1219,1588.023401,64.30635,89.903848,85.15625,93.75,59.375,65.697674,1166.245217,44.854093,60.938954,59.592109,82.620009,42.050841,54.299599,513.940069,452.925154
3,Austria,2011,1321,2788.674561,66.906092,72.863248,60.0625,81.25,75.0,45.348837,2632.194486,96.736029,105.282028,101.218176,97.872116,94.529457,57.58021,428.827792,521.19203
4,Austria,2011,1324,1799.962646,65.660399,87.948718,83.75,87.5,62.5,50.0,1804.782956,67.982449,94.822467,85.356768,43.819472,64.502054,52.969386,468.942539,234.843595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4268,Spain,2013,9334,1046.085136,39.659487,81.911422,82.545455,84.363636,46.985294,46.678593,1092.568983,43.613822,84.228471,86.14677,88.254037,51.95402,38.838715,369.81613,310.287247
4269,Spain,2013,2100,2069.535937,66.338245,88.46154,98.75,97.5,57.5,59.069767,3654.135875,115.455341,154.932298,171.49274,170.467285,103.722052,84.230581,467.619552,800.300298
4270,Spain,2013,3258,1435.902368,67.310957,71.28205,68.7,96.363636,57.5,52.55814,1456.858411,72.508648,76.626674,70.518447,100.578868,68.131395,61.410752,409.30079,471.574849
4271,Spain,2013,4212,1261.978491,50.317439,78.076923,75.3875,93.522727,67.5,41.860465,782.891747,38.370388,54.05148,52.370037,74.793264,44.732223,28.959014,418.935822,309.266499


In [20]:
pd.read_csv(
    "/Users/alexandralugova/Documents/GitHub/MH-old-workers/data/datasets/results/work_quality_indexes_year_country.csv"
)

,country,year,isco,jqi_monthly_earnings,jqi_skills_discretion,jqi_physical_environment,jqi_working_time_quality,jqi_social_environment,jqi_prospects,jqi_intensity,jqi_monthly_earnings_weighted,jqi_skills_discretion_weighted,jqi_physical_environment_weighted,jqi_working_time_quality_weighted,jqi_social_environment_weighted,jqi_prospects_weighted,jqi_intensity_weighted,jqi_sum,jqi_sum_weighted
0,Austria,2011,1114,2366.148112,77.290234,97.863248,100.000000,100.000000,75.000000,52.325581,3034.577207,94.406660,121.392346,123.996761,60.311810,45.233857,47.096205,523.975161,316.018902
1,Austria,2011,1120,3143.596761,83.616787,85.042736,53.125000,87.500000,78.571429,53.820598,1638.952541,52.464895,53.989173,32.531523,47.019517,49.624036,34.312300,462.649600,250.798543
2,Austria,2011,1219,1588.023401,64.306350,89.903848,85.156250,93.750000,59.375000,65.697674,1166.245217,44.854093,60.938954,59.592109,82.620009,42.050841,54.299599,513.940069,452.925154
3,Austria,2011,1321,2788.674561,66.906092,72.863248,60.062500,81.250000,75.000000,45.348837,2632.194486,96.736029,105.282028,101.218176,97.872116,94.529457,57.580210,428.827792,521.192030
4,Austria,2011,1324,1799.962646,65.660399,87.948718,83.750000,87.500000,62.500000,50.000000,1804.782956,67.982449,94.822467,85.356768,43.819472,64.502054,52.969386,468.942539,234.843595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4268,Spain,2013,9334,1046.085136,39.659487,81.911422,82.545455,84.363636,46.985294,46.678593,1092.568983,43.613822,84.228471,86.146770,88.254037,51.954020,38.838715,369.816130,310.287247
4269,Spain,2013,2100,2069.535937,66.338245,88.461540,98.750000,97.500000,57.500000,59.069767,3654.135875,115.455341,154.932298,171.492740,170.467285,103.722052,84.230581,467.619552,800.300298
4270,Spain,2013,3258,1435.902368,67.310957,71.282050,68.700000,96.363636,57.500000,52.558140,1456.858411,72.508648,76.626674,70.518447,100.578868,68.131395,61.410752,409.300790,471.574849
4271,Spain,2013,4212,1261.978491,50.317439,78.076923,75.387500,93.522727,67.500000,41.860465,782.891747,38.370388,54.051480,52.370037,74.793264,44.732223,28.959014,418.935822,309.266499
